# Personalization & Ranking Demo

This notebook generates synthetic user-item interactions, trains a simple GBDT baseline, and produces a ranked list of candidates with scores.

In [ ]:
import pandas as pd
from src.common.synthetic_data import generate_synthetic_interactions

interactions = generate_synthetic_interactions(n=30000)
interactions.head()

In [ ]:
from sklearn.model_selection import train_test_split
from src.personalization.ranking import train_gbdt_ranker, rank_candidates

# Create a tiny feature set: user and item hash buckets
X = pd.DataFrame({
    'user_bucket': interactions['user_hash'].apply(lambda s: int(s.split('_')[1]) % 1000),
    'item_bucket': interactions['item_id'].apply(lambda s: int(s.split('_')[1]) % 1000),
})
y = interactions['clicked']

Xtr, Xte, ytr, yte, item_te = train_test_split(X, y, interactions['item_id'], test_size=0.2, random_state=42)
model = train_gbdt_ranker(Xtr, ytr)
ranked = rank_candidates(model, Xte, item_te)
ranked.head(10)